In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df=pd.read_csv(r"Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
df=df.drop(["Id","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Time","Summary"],axis="columns")
df.head()

,ProductId,UserId,Score,Text
0,B001E4KFG0,A3SGXH7AUHU8GW,5,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,1,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,4,This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,2,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy at a great price. There was a wid...


In [4]:
df.isna().sum()

ProductId    0
UserId       0
Score        0
Text         0
dtype: int64

In [5]:
from scipy.sparse import csr_matrix
data_pivot=df[:10000].pivot_table(index='ProductId',columns='UserId',values='Score').fillna(0)
data_pivot_matrix=csr_matrix(data_pivot.values)

In [6]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric='cosine')
model.fit(data_pivot_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [7]:
data_index = list(data_pivot.index)

In [8]:
item_id_index = data_index.index('B001E4KFG0')

In [9]:
distance,indices=model.kneighbors(data_pivot.iloc[item_id_index,:].values.reshape(1,-1))

In [10]:
dist=distance.reshape(-1)
ind=indices.reshape(-1)

In [11]:
item_id='B001E4KFG0'
recommend_data=dict()
lst=[]
for i in range(1,len(dist)):
    bid=data_pivot.index[ind[i]]
    recommend_data[str(bid)]=float(dist[i])
    print(f'Distance of dish with id {bid} with respect to dish with id {item_id}={recommend_data[str(bid)]}')
    lst.append(df.loc[df['ProductId']==bid]['ProductId'].values[0])

Distance of dish with id B003TIVBSU with respect to dish with id B001E4KFG0=1.0
Distance of dish with id B003TQQKFQ with respect to dish with id B001E4KFG0=1.0
Distance of dish with id B003U9TAH2 with respect to dish with id B001E4KFG0=1.0
Distance of dish with id B003TIUM5S with respect to dish with id B001E4KFG0=1.0


In [12]:
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
def sentiment_calc(text):
    try:
        return TextBlob(str(text)).sentiment.polarity
    except:
        return None
df['sentiment']=df['Text'].apply(sentiment_calc)
df.head()

,ProductId,UserId,Score,Text,sentiment
0,B001E4KFG0,A3SGXH7AUHU8GW,5,I have bought several of the Vitality canned d...,0.450000
1,B00813GRG4,A1D87F6ZCVE5NK,1,Product arrived labeled as Jumbo Salted Peanut...,-0.033333
2,B000LQOCH0,ABXLMWJIXXAIN,4,This is a confection that has been around a fe...,0.133571
3,B000UA0QIQ,A395BORC6FGVXV,2,If you are looking for the secret ingredient i...,0.166667
4,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy at a great price. There was a wid...,0.483333


In [14]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['UserId']=le.fit_transform(df['UserId'])
df

,ProductId,UserId,Score,Text,sentiment
0,B001E4KFG0,188646,5,I have bought several of the Vitality canned d...,0.450000
1,B00813GRG4,25105,1,Product arrived labeled as Jumbo Salted Peanut...,-0.033333
2,B000LQOCH0,210482,4,This is a confection that has been around a fe...,0.133571
3,B000UA0QIQ,152635,2,If you are looking for the secret ingredient i...,0.166667
4,B006K2ZZ7K,57804,5,Great taffy at a great price. There was a wid...,0.483333
...,...,...,...,...,...
568449,B001EO7N10,83892,5,Great for sesame chicken..this is a good if no...,0.365000
568450,B003S1WTCU,169568,2,I'm disappointed with the flavor. The chocolat...,-0.208333
568451,B004I613EE,4347,5,"These stars are small, so you can give 10-15 o...",-0.021875
568452,B004I613EE,169728,5,These are the BEST treats for training and rew...,0.517857


In [15]:
def classify_scores(a):
    if a<=-0.25:
        v=1
    elif a<=1:
        v=2
    elif a<=2:
        v=3
    elif a<=3:
        v=4
    elif a<=5:
        v=5
    return v
df['New_score']=df['Score'].apply(classify_scores)
df

,ProductId,UserId,Score,Text,sentiment,New_score
0,B001E4KFG0,188646,5,I have bought several of the Vitality canned d...,0.450000,5
1,B00813GRG4,25105,1,Product arrived labeled as Jumbo Salted Peanut...,-0.033333,2
2,B000LQOCH0,210482,4,This is a confection that has been around a fe...,0.133571,5
3,B000UA0QIQ,152635,2,If you are looking for the secret ingredient i...,0.166667,3
4,B006K2ZZ7K,57804,5,Great taffy at a great price. There was a wid...,0.483333,5
...,...,...,...,...,...,...
568449,B001EO7N10,83892,5,Great for sesame chicken..this is a good if no...,0.365000,5
568450,B003S1WTCU,169568,2,I'm disappointed with the flavor. The chocolat...,-0.208333,3
568451,B004I613EE,4347,5,"These stars are small, so you can give 10-15 o...",-0.021875,5
568452,B004I613EE,169728,5,These are the BEST treats for training and rew...,0.517857,5


In [16]:
df['Updated_score'] = df['Score']*df['sentiment']
df

,ProductId,UserId,Score,Text,sentiment,New_score,Updated_score
0,B001E4KFG0,188646,5,I have bought several of the Vitality canned d...,0.450000,5,2.250000
1,B00813GRG4,25105,1,Product arrived labeled as Jumbo Salted Peanut...,-0.033333,2,-0.033333
2,B000LQOCH0,210482,4,This is a confection that has been around a fe...,0.133571,5,0.534286
3,B000UA0QIQ,152635,2,If you are looking for the secret ingredient i...,0.166667,3,0.333333
4,B006K2ZZ7K,57804,5,Great taffy at a great price. There was a wid...,0.483333,5,2.416667
...,...,...,...,...,...,...,...
568449,B001EO7N10,83892,5,Great for sesame chicken..this is a good if no...,0.365000,5,1.825000
568450,B003S1WTCU,169568,2,I'm disappointed with the flavor. The chocolat...,-0.208333,3,-0.416667
568451,B004I613EE,4347,5,"These stars are small, so you can give 10-15 o...",-0.021875,5,-0.109375
568452,B004I613EE,169728,5,These are the BEST treats for training and rew...,0.517857,5,2.589286


In [17]:
df_pivot = df[:10000].pivot_table(index='ProductId',columns='UserId',values='New_score').fillna(0)
df_pivot

UserId,509,512,533,595,698,707,718,722,750,810,...,255725,255729,255806,255836,255837,255866,255901,255916,255988,256019
ProductId,,,,,,,,,,,,,,,,,,,,,
B00002NCJC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002Z754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005V3DC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000084DVR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000084E1U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B009AVDFCK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B009D668KS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B009HINRX8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_pivot = df[:1000].pivot_table(index='ProductId',columns='UserId',values='New_score').fillna(0)
df_pivot

UserId,810,1025,1339,1535,2004,3542,3821,4023,4045,4073,...,252859,252901,253461,253784,254168,254289,254334,254531,255302,255461
ProductId,,,,,,,,,,,,,,,,,,,,,
B0001PB9FE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0001PB9FY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0002567IW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00029XIZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0002XIB2Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0087HW5E2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0089PI9OC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B008BEGP9W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_pivot = df[:1000].pivot_table(index='ProductId',columns='UserId',values='New_score').fillna(0)
df_pivot

UserId,810,1025,1339,1535,2004,3542,3821,4023,4045,4073,...,252859,252901,253461,253784,254168,254289,254334,254531,255302,255461
ProductId,,,,,,,,,,,,,,,,,,,,,
B0001PB9FE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0001PB9FY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0002567IW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00029XIZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0002XIB2Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0087HW5E2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0089PI9OC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B008BEGP9W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_pivot = df[:10].pivot_table(index='ProductId',columns='UserId',values='New_score').fillna(0)
df_pivot

UserId,25105,43414,54048,57804,70264,152635,172409,188646,210482,213950
ProductId,,,,,,,,,,
B000E7L2R4,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000LQOCH0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
B000UA0QIQ,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
B00171APVA,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
B001E4KFG0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
B006K2ZZ7K,0.0,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0
B00813GRG4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from scipy.sparse import csr_matrix

df_pivot_matrix = csr_matrix(df_pivot.values) 
print(df_pivot_matrix)

  (0, 1)	5.0
  (1, 8)	5.0
  (2, 5)	3.0
  (3, 4)	5.0
  (4, 7)	5.0
  (5, 2)	5.0
  (5, 3)	5.0
  (5, 6)	5.0
  (5, 9)	5.0
  (6, 0)	2.0


In [22]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', n_neighbors=20, radius=1)
model_knn.fit(df_pivot_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1)

In [23]:
similarity_matrix = cosine_similarity(df_pivot)
similarity_matrix

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [24]:
item_id='B001E4KFG0'
data = list(df_pivot.index) #shows list of ProductID in data-set
print(data)

['B000E7L2R4', 'B000LQOCH0', 'B000UA0QIQ', 'B00171APVA', 'B001E4KFG0', 'B006K2ZZ7K', 'B00813GRG4']


In [25]:
query_index = data.index(item_id) #shows index of productID by USER
print(query_index)

4


In [26]:
similarity, indices = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)
print(similarity) #shows similarity distance through productID by USER
print(indices)

[[0. 1. 1. 1. 1.]]
[[4 3 5 0 2]]


In [27]:
data_dict={}
for i in range(0, len(similarity.flatten())):    #gives length of similarity array

    if i == 0:
        print('Recommendations for {0}:\n'.format(df_pivot.index[query_index]))
    else:
        data_dict[str(df_pivot.index[indices.flatten()[i]])] = float(similarity.flatten()[i])
        print(f'{df_pivot.index[indices.flatten()[i]]}, is similarity distance = with {similarity.flatten()[i]:.20f}:')

print(data_dict)

Recommendations for B001E4KFG0:

B00171APVA, is similarity distance = with 1.00000000000000000000:
B006K2ZZ7K, is similarity distance = with 1.00000000000000000000:
B000E7L2R4, is similarity distance = with 1.00000000000000000000:
B000UA0QIQ, is similarity distance = with 1.00000000000000000000:
{'B00171APVA': 1.0, 'B006K2ZZ7K': 1.0, 'B000E7L2R4': 1.0, 'B000UA0QIQ': 1.0}
